# 필요 라이브러리

In [1]:
import warnings
import os
import pandas as pd

warnings.filterwarnings('ignore')

In [2]:
print(os.getcwd()) # 현재 경로
# print(os.listdir(os.getcwd())) # 현재 경로 파일

os.chdir('/project/kcure-08/src/1. Data/1. Colon') # 현재 경로 변경(데이터 파일)
print(os.getcwd())
# print(os.listdir(os.getcwd()))

/project/kcure-08/src/2. Model/1. Colon
/project/kcure-08/src/1. Data/1. Colon


# 1. 데이터 불러오기

In [6]:
### raw 데이터 불러오기(확인용) ### 
df_cc_t200 = pd.read_csv('./Raw_csv/cs_cc_t200.csv') # 진료(명세서)
df_cc_t400 = pd.read_csv('./Raw_csv/cs_cc_t400.csv') # 진료(상병내역)

In [ ]:
### 진료테이블 불러오기(t200 & t400)
df_cs_cc = pd.read_csv('./PP_csv/cs_cc.csv')
df_cs_cc # 21897420 rows × 12 columns

-----

# 2. 데이터 전처리

### U99 주/부상병 코드 U99 시점 진료 시작일자 찾기

In [ ]:
### 진료테이블 T200: 
df_cs_cc_rec = df_cs_cc[((df_cs_cc['SICK_CD_NEW'] == 'U99')| (df_cs_cc['M_SICK_CD'] == 'U99') | (df_cs_cc['SUB_SICK_CD'] == 'U99')) & (df_cs_cc['SICK_TY_CD'] != 3.0)]

### 중복값 제거 및 첫번째 값 반환
df_cs_cc_rec = df_cs_cc_rec.drop_duplicates('SN_KEY') 

In [ ]:
df_cs_cc_rec['TIME_OF_REC_END'] = df_cs_cc_rec['RECU_FR_DD']
index_dp = df_cs_cc_rec[df_cs_cc_rec['SN_KEY'] == 'SN00042239'].index ### SN00042239 찾는 코드는 아래에
df_cs_cc_rec = df_cs_cc_rec.drop(index_dp)

-----

### U99 이전 첫 진료 종료일자 찾기 

In [ ]:
df_cs_cc_tmp = df_cs_cc.drop_duplicates('MID') # 8622521 rows × 12 columns

In [ ]:
### U99 코드를 가진 피험자 기준으로 데이터 가져오기
cust_id = list(df_cs_cc_rec['SN_KEY'])
df_cs_cc_tmp = df_cs_cc_tmp[df_cs_cc_tmp['SN_KEY'].isin(cust_id)] # 75052 rows × 12 columns

In [ ]:
df_cs_cc_tmp = df_cs_cc_tmp[(df_cs_cc_tmp['SICK_CD_NEW'] == 'C18') | (df_cs_cc_tmp['SICK_CD_NEW'] == 'C19') | (df_cs_cc_tmp['SICK_CD_NEW'] == 'C20')| (df_cs_cc_tmp['M_SICK_CD'] == 'C18') | (df_cs_cc_tmp['M_SICK_CD'] == 'C19') | (df_cs_cc_tmp['M_SICK_CD'] == 'C20') | (df_cs_cc_tmp['SUB_SICK_CD'] == 'C18') | (df_cs_cc_tmp['SUB_SICK_CD'] == 'C19') | (df_cs_cc_tmp['SUB_SICK_CD'] == 'C20')]
df_cs_cc_tmp # 12760 rows × 12 columns

In [ ]:
df_cs_cc_tmp = df_cs_cc_tmp.drop_duplicates('SN_KEY') 
df_cs_cc_tmp # 194 rows × 12 columns

In [ ]:
TIME_OF_REC_START = df_cs_cc_tmp['RECU_TO_DD']
TIME_OF_REC_START = list(TIME_OF_REC_START)

In [40]:
df_cs_cc_rec['TIME_OF_REC_START'] = TIME_OF_REC_START

-----

# 전처리 후 변수 저장하기

In [42]:
df_cs_cc_rec_fin = df_cs_cc_rec[['SN_KEY', 'TIME_OF_REC_START', 'TIME_OF_REC_END']]

In [43]:
df_cs_cc_rec_fin['TIME_OF_REC_START'] = pd.to_datetime(df_cs_cc_rec_fin['TIME_OF_REC_START'], format = '%Y-%m-%d', errors='raise')
df_cs_cc_rec_fin['TIME_OF_REC_END'] = pd.to_datetime(df_cs_cc_rec_fin['TIME_OF_REC_END'], format = '%Y-%m-%d', errors='raise')

In [44]:
df_cs_cc_rec_fin['TIME_OF_REC'] = df_cs_cc_rec_fin['TIME_OF_REC_END'] - df_cs_cc_rec_fin['TIME_OF_REC_START']

In [47]:
df_cs_cc_rec_fin['TIME_OF_REC'].value_counts()

523 days     3
349 days     2
577 days     2
477 days     2
1525 days    2
            ..
467 days     1
1642 days    1
1350 days    1
103 days     1
21 days      1
Name: TIME_OF_REC, Length: 179, dtype: int64

In [ ]:
df_cs_cc_rec_fin[df_cs_cc_rec_fin['TIME_OF_REC'] < '1825days']

------------------

끝